In [ ]:
#Essential Libraries

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
print(os.listdir("../content/drive/MyDrive/dataset/archive/data/train"))

from fastai.vision import *
from fastai.metrics import error_rate
# Any results you write to the current directory are saved as output.

#start batch size at 64 can reduce based on GPU
bs = 64

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
#Dataset

In [ ]:
img_dir='../content/drive/MyDrive/dataset/archive/data/train'
path=Path(img_dir)
path

In [ ]:
data = ImageDataBunch.from_folder(path, train=".", 
                                  valid_pct=0.2,
                                  ds_tfms=get_transforms(flip_vert=True, max_warp=0),
                                  size=224,bs=64, 
                                  num_workers=0).normalize(imagenet_stats)

In [ ]:
data.show_batch(rows=3, figsize=(7,6))

In [ ]:
#Model ResNet34

In [ ]:
learn = cnn_learner(data, models.resnet34, metrics=accuracy, model_dir="/tmp/model/")

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(30,1e-2)

In [ ]:
learn.save('stage-1')

In [ ]:
#Check Model

In [ ]:
interp = ClassificationInterpretation.from_learner(learn)

losses,idxs = interp.top_losses()

len(data.valid_ds)==len(losses)==len(idxs)

In [ ]:
interp.plot_top_losses(9, figsize=(15,11))

In [ ]:
interp.plot_confusion_matrix(figsize=(4,4), dpi=60)

In [ ]:
#Optimize ResNet34

In [ ]:
learn.unfreeze()

In [ ]:
learn.lr_find()
learn.recorder.plot()

In [ ]:
learn.unfreeze()
learn.fit_one_cycle(10, max_lr=slice(1e-6,1e-4))